In [175]:
# import libraries
import pandas as pd
from sspipe import p, px
import texthero as hero
from texthero import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import re

In [3]:
# read files
df_prod_tech_specs = pd.read_csv("data/products_technical_specs.csv")
df_prod_titles = pd.read_csv("data/products_titles.csv")

# 1. Prepare and clean data

## titles

In [4]:
df_prod_titles

,Product_No,Product_title
0,1000,Renkforce Strom Verlängerungskabel [1x IDE-Str...
1,1001,Renkforce Strom Anschlusskabel [1x ATX-Strom-S...
2,1002,Akasa Strom Adapter [2x SATA-Strom-Stecker 15p...
3,1003,LogiLink Kaltgeräte Anschlusskabel [1x Schutzk...
4,1004,Digitus RJ45 Netzwerk Anschlusskabel CAT 5e U/...
5,1006,Renkforce RJ45 Netzwerk Verlängerungskabel CAT...
6,1005,BKL Electronic 073331 Kaltgeräte-Adapter Kaltg...
7,1007,Netz-Anschlusskabel Kleeblatt-Buchse C5 - Kabe...
8,1008,Neutrik NKFCA30-0 Netz-Anschlusskabel PowerCon...
9,1009,SIROX 346.310.04 Strom Verlängerungskabel 16 A...


In [5]:
df_prod_titles.info()
# => 26 rows, no nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product_No     26 non-null     int64 
 1   Product_title  26 non-null     object
dtypes: int64(1), object(1)
memory usage: 544.0+ bytes


In [6]:
# check every column for duplicates
[df_prod_titles[col].unique().size for col in df_prod_titles]
# => no duplicates

[26, 26]

## specs

In [7]:
df_prod_tech_specs
# => Product_tech_specs contains multiple information (split by | and create extra columns)

,Product_No,Product_tech_specs
0,1000,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...
1,1001,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...
2,1002,"Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ..."
3,1003,Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....
4,1004,AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...
5,1005,Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...
6,1007,Anschlusstyp A (bzw. Eingänge): Kleeblatt-...
7,1008,Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...
8,1009,Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...
9,1006,AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...


In [8]:
df_prod_tech_specs.info()
# => 26 rows, no nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Product_No          26 non-null     int64 
 1   Product_tech_specs  26 non-null     object
dtypes: int64(1), object(1)
memory usage: 544.0+ bytes


In [9]:
# check every column for duplicates
[df_prod_tech_specs[col].unique().size for col in df_prod_tech_specs]
# => no duplicates

[26, 26]

In [109]:
# create columns for specs

def check_for_two_elements(arr):
    """ Check for a length of exactly 2 """
    if len(arr) != 2:
        raise Exception("Data in wrong format", arr)
    return arr

# Im using pipe notation here, which is enabled by sspipe (more info here: https://github.com/sspipe/sspipe)
df_new_spec_columns = (
    df_prod_tech_specs["Product_tech_specs"]
    | p(map, (
        p(re.sub, r"\|+", "|", px)          # remove repeating and trailing delimiters
        | px.strip("|")
        | p(re.sub, r"\|Löt", ", Löt", px)  # Fix one product that uses | in a spec
        | px.split("|")                     # split into single specs (e.g. "AWG: 18")
        | p(map, (
            px.split(':', 1)                # split single specs into name, value pairs (e.g. ["AWG", "18"])
            | p(map, px.strip(" ,"))        # trim trailing spaces and comma
            | p(list)
            | p(check_for_two_elements)     # check, that there are exactly two elements (name, value)
        ))
        | p(pd.DataFrame)
        | px.set_index(0)
        | px.transpose()
    ))
    | p(pd.concat)
    | px.reset_index(drop=True)
)

print("New columns shape: ", df_new_spec_columns.shape)
# df_new_spec_columns

df_prod_tech_specs_parsed = df_prod_tech_specs.merge(df_new_spec_columns, left_index=True, right_index=True)
df_prod_tech_specs_parsed
# => it later turned out this wasn't necessary...

New columns shape:  (26, 82)


,Product_No,Product_tech_specs,AWG,Abschirmung,Anschluss-Typ,Anschlusstyp A (bzw. Eingänge),Anschlusstyp B (bzw. Ausgänge),Anzahl Anschluss A,Anzahl Anschluss B,Herstellerfarbe,...,"Produktabmessung, Tiefe",Ausführung (Gewindebohrer),Drehrichtung Gewinde,Gewinde-Länge,Gewinde-Maß,Gewindeart,Kernloch-Größe,Schaftvierkant,Steigung,Anschnittform
0,1000,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,IDE,IDE-Strom-Stecker 4pol.,IDE-Strom-Buchse 4pol.,1 x,2 x,"Schwarz, Rot, Gelb",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,Mainboard,ATX-Strom-Stecker 14pol.,ATX-Strom-Buchse 24pol.,1 x,1 x,Bunt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,"Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ...",NaN,NaN,"SATA, PCI-Express",SATA-Strom-Stecker 15pol.,PCIe-Stecker 6pol.,2 x,1 x,"Schwarz, Gelb",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....,NaN,NaN,Kaltgeräte,Schutzkontakt-Stecker,Kaltgeräte-Buchse C13,1 x,1 x,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Stecker,1 x,1 x,Gelb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1005,Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...,NaN,NaN,NaN,Kaltgeräte-Stecker C14,"Kaltgeräte-Buchse C13, Kaltgeräte-Buchse C13",NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1007,Anschlusstyp A (bzw. Eingänge): Kleeblatt-...,NaN,NaN,NaN,NaN,offene Kabelenden,NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1008,Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...,NaN,NaN,NaN,PowerCon-Buchse,offene Kabelenden,NaN,NaN,Schwarz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1009,Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...,NaN,NaN,NaN,Schutzkontakt-Gummi-Stecker,Schutzkontakt-Gummi-Kupplung,NaN,NaN,Rot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1006,AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Buchse,1 x,1 x,Violett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Combine datasets

In [98]:
# check, if all ids match
print("If all are true, all product ids are present in both datasets.")
df_prod_titles["Product_No"].sort_values().reset_index(drop=True) == df_prod_tech_specs["Product_No"].sort_values().reset_index(drop=True)
# => they do (no False)

If all are true, all product ids are present in both datasets.


0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
Name: Product_No, dtype: bool

In [95]:
df_prod = df_prod_titles.merge(df_prod_tech_specs_parsed, on="Product_No")
df_prod

,Product_No,Product_title,Product_tech_specs,AWG,Abschirmung,Anschluss-Typ,Anschlusstyp A (bzw. Eingänge),Anschlusstyp B (bzw. Ausgänge),Anzahl Anschluss A,Anzahl Anschluss B,...,"Produktabmessung, Tiefe",Ausführung (Gewindebohrer),Drehrichtung Gewinde,Gewinde-Länge,Gewinde-Maß,Gewindeart,Kernloch-Größe,Schaftvierkant,Steigung,Anschnittform
0,1000,Renkforce Strom Verlängerungskabel [1x IDE-Str...,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,IDE,IDE-Strom-Stecker 4pol.,IDE-Strom-Buchse 4pol.,1 x,2 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,Renkforce Strom Anschlusskabel [1x ATX-Strom-S...,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,18,ohne Schirmung,Mainboard,ATX-Strom-Stecker 14pol.,ATX-Strom-Buchse 24pol.,1 x,1 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,Akasa Strom Adapter [2x SATA-Strom-Stecker 15p...,"Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ...",NaN,NaN,"SATA, PCI-Express",SATA-Strom-Stecker 15pol.,PCIe-Stecker 6pol.,2 x,1 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,LogiLink Kaltgeräte Anschlusskabel [1x Schutzk...,Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....,NaN,NaN,Kaltgeräte,Schutzkontakt-Stecker,Kaltgeräte-Buchse C13,1 x,1 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,Digitus RJ45 Netzwerk Anschlusskabel CAT 5e U/...,AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Stecker,1 x,1 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1006,Renkforce RJ45 Netzwerk Verlängerungskabel CAT...,AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Buchse,1 x,1 x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1005,BKL Electronic 073331 Kaltgeräte-Adapter Kaltg...,Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...,NaN,NaN,NaN,Kaltgeräte-Stecker C14,"Kaltgeräte-Buchse C13, Kaltgeräte-Buchse C13",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1007,Netz-Anschlusskabel Kleeblatt-Buchse C5 - Kabe...,Anschlusstyp A (bzw. Eingänge): Kleeblatt-...,NaN,NaN,NaN,NaN,offene Kabelenden,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1008,Neutrik NKFCA30-0 Netz-Anschlusskabel PowerCon...,Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...,NaN,NaN,NaN,PowerCon-Buchse,offene Kabelenden,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1009,SIROX 346.310.04 Strom Verlängerungskabel 16 A...,Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...,NaN,NaN,NaN,Schutzkontakt-Gummi-Stecker,Schutzkontakt-Gummi-Kupplung,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Calculate all product similarities via TF-IDF

Here we will
 1. Clean title and specs.
 2. Vectorize title and text with TF-IDF.
 3. Calculate the cosine similarity.

In [141]:
# Clean title and specs

# this is the default pipeline from texthero, but uses german stopwords and removes quantity tokens like 2x (they make no sense in word bag algorithms)
clean_pipeline = [
    preprocessing.fillna,
    preprocessing.lowercase,
    preprocessing.remove_digits,
    preprocessing.remove_punctuation,
    preprocessing.remove_diacritics,
    lambda s: preprocessing.remove_stopwords(s, stopwords=stopwords.words("german")),
    lambda s: pd.Series([re.sub(r"\d{0,2}x", "", e) for e in s]),
    preprocessing.remove_whitespace,
]

clean_title = hero.clean(df_prod["Product_title"], clean_pipeline)
clean_specs = hero.clean(df_prod["Product_tech_specs"], clean_pipeline)

df_prod_cleaned = df_prod.copy()
df_prod_cleaned.insert(2, 'Product_title_clean', clean_title)
df_prod_cleaned.insert(4, 'Product_tech_specs_clean', clean_specs)
df_prod_cleaned

,Product_No,Product_title,Product_title_clean,Product_tech_specs,Product_tech_specs_clean,AWG,Abschirmung,Anschluss-Typ,Anschlusstyp A (bzw. Eingänge),Anschlusstyp B (bzw. Ausgänge),...,"Produktabmessung, Tiefe",Ausführung (Gewindebohrer),Drehrichtung Gewinde,Gewinde-Länge,Gewinde-Maß,Gewindeart,Kernloch-Größe,Schaftvierkant,Steigung,Anschnittform
0,1000,Renkforce Strom Verlängerungskabel [1x IDE-Str...,renkforce strom verlangerungskabel ide strom s...,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,awg abschirmung schirmung anschluss typ ide an...,18,ohne Schirmung,IDE,IDE-Strom-Stecker 4pol.,IDE-Strom-Buchse 4pol.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,Renkforce Strom Anschlusskabel [1x ATX-Strom-S...,renkforce strom anschlusskabel at strom stecke...,AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,awg abschirmung schirmung anschluss typ mainbo...,18,ohne Schirmung,Mainboard,ATX-Strom-Stecker 14pol.,ATX-Strom-Buchse 24pol.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,Akasa Strom Adapter [2x SATA-Strom-Stecker 15p...,akasa strom adapter sata strom stecker 15pol p...,"Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ...",anschluss typ sata pci epress anschlusstyp a b...,NaN,NaN,"SATA, PCI-Express",SATA-Strom-Stecker 15pol.,PCIe-Stecker 6pol.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,LogiLink Kaltgeräte Anschlusskabel [1x Schutzk...,logilink kaltgerate anschlusskabel schutzkonta...,Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....,anschluss typ kaltgerate anschlusstyp a bzw ei...,NaN,NaN,Kaltgeräte,Schutzkontakt-Stecker,Kaltgeräte-Buchse C13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,Digitus RJ45 Netzwerk Anschlusskabel CAT 5e U/...,digitus rj45 netzwerk anschlusskabel cat 5e u ...,AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...,awg anschluss typ rj45 anschlusstyp a bzw eing...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Stecker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1006,Renkforce RJ45 Netzwerk Verlängerungskabel CAT...,renkforce rj45 netzwerk verlangerungskabel cat...,AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...,awg anschluss typ rj45 anschlusstyp a bzw eing...,26/7,NaN,RJ45,RJ45-Stecker,RJ45-Buchse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1005,BKL Electronic 073331 Kaltgeräte-Adapter Kaltg...,bkl electronic kaltgerate adapter kaltgerate s...,Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...,anschlusstyp a bzw eingange kaltgerate stecker...,NaN,NaN,NaN,Kaltgeräte-Stecker C14,"Kaltgeräte-Buchse C13, Kaltgeräte-Buchse C13",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1007,Netz-Anschlusskabel Kleeblatt-Buchse C5 - Kabe...,netz anschlusskabel kleeblatt buchse c5 kabel ...,Anschlusstyp A (bzw. Eingänge): Kleeblatt-...,anschlusstyp a bzw eingange kleeblatt buchse c...,NaN,NaN,NaN,NaN,offene Kabelenden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1008,Neutrik NKFCA30-0 Netz-Anschlusskabel PowerCon...,neutrik nkfca30 netz anschlusskabel powercon b...,Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...,anschlusstyp a bzw eingange powercon buchse an...,NaN,NaN,NaN,PowerCon-Buchse,offene Kabelenden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1009,SIROX 346.310.04 Strom Verlängerungskabel 16 A...,siro strom verlangerungskabel a rot m,Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...,anschlusstyp a bzw eingange schutzkontakt gumm...,NaN,NaN,NaN,Schutzkontakt-Gummi-Stecker,Schutzkontakt-Gummi-Kupplung,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
# vectorize title and specs with TF-IDF

tfidf_title = hero.tfidf(df_prod_cleaned["Product_title_clean"])
tfidf_specs = hero.tfidf(df_prod_cleaned["Product_tech_specs_clean"])

df_prod_vectorized = df_prod_cleaned.copy()
df_prod_vectorized.insert(3, 'Product_title_tfidf', tfidf_title)
df_prod_vectorized.insert(6, 'Product_tech_specs_tfidf', tfidf_specs)
df_prod_vectorized
# hero.scatterplot(df_prod_vectorized, col='Product_title_tfidf', hover_data=["Product_title"])

,Product_No,Product_title,Product_title_clean,Product_title_tfidf,Product_tech_specs,Product_tech_specs_clean,Product_tech_specs_tfidf,AWG,Abschirmung,Anschluss-Typ,...,"Produktabmessung, Tiefe",Ausführung (Gewindebohrer),Drehrichtung Gewinde,Gewinde-Länge,Gewinde-Maß,Gewindeart,Kernloch-Größe,Schaftvierkant,Steigung,Anschnittform
0,1000,Renkforce Strom Verlängerungskabel [1x IDE-Str...,renkforce strom verlangerungskabel ide strom s...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.476...",AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,awg abschirmung schirmung anschluss typ ide an...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28798477...",18,ohne Schirmung,IDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,Renkforce Strom Anschlusskabel [1x ATX-Strom-S...,renkforce strom anschlusskabel at strom stecke...,"[0.0, 0.0, 0.26167567477377623, 0.0, 0.2616756...",AWG: 18|Abschirmung: ohne Schirmung|Anschluss-...,awg abschirmung schirmung anschluss typ mainbo...,"[0.0, 0.15667965306238205, 0.0, 0.156679653062...",18,ohne Schirmung,Mainboard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,Akasa Strom Adapter [2x SATA-Strom-Stecker 15p...,akasa strom adapter sata strom stecker 15pol p...,"[0.0, 0.0, 0.0, 0.29389202578095047, 0.0, 0.0,...","Anschluss-Typ: SATA, PCI-Express|Anschlusstyp ...",anschluss typ sata pci epress anschlusstyp a b...,"[0.0, 0.0, 0.15538697268732848, 0.0, 0.0, 0.0,...",NaN,NaN,"SATA, PCI-Express",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,LogiLink Kaltgeräte Anschlusskabel [1x Schutzk...,logilink kaltgerate anschlusskabel schutzkonta...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Anschluss-Typ: Kaltgeräte|Anschlusstyp A (bzw....,anschluss typ kaltgerate anschlusstyp a bzw ei...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,Kaltgeräte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1004,Digitus RJ45 Netzwerk Anschlusskabel CAT 5e U/...,digitus rj45 netzwerk anschlusskabel cat 5e u ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",AWG: 26/7|Anschluss-Typ: RJ45|Anschlusstyp A (...,awg anschluss typ rj45 anschlusstyp a bzw eing...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.143...",26/7,NaN,RJ45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1006,Renkforce RJ45 Netzwerk Verlängerungskabel CAT...,renkforce rj45 netzwerk verlangerungskabel cat...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",AWG: 26/7|Anschluss-Typ: RJ45|||Anschlusstyp A...,awg anschluss typ rj45 anschlusstyp a bzw eing...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",26/7,NaN,RJ45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1005,BKL Electronic 073331 Kaltgeräte-Adapter Kaltg...,bkl electronic kaltgerate adapter kaltgerate s...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Anschlusstyp A (bzw. Eingänge): Kaltgeräte-Ste...,anschlusstyp a bzw eingange kaltgerate stecker...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1007,Netz-Anschlusskabel Kleeblatt-Buchse C5 - Kabe...,netz anschlusskabel kleeblatt buchse c5 kabel ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Anschlusstyp A (bzw. Eingänge): Kleeblatt-...,anschlusstyp a bzw eingange kleeblatt buchse c...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1008,Neutrik NKFCA30-0 Netz-Anschlusskabel PowerCon...,neutrik nkfca30 netz anschlusskabel powercon b...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Anschlusstyp A (bzw. Eingänge): PowerCon-Buchs...,anschlusstyp a bzw eingange powercon buchse an...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1009,SIROX 346.310.04 Strom Verlängerungskabel 16 A...,siro strom verlangerungskabel a rot m,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Anschlusstyp A (bzw. Eingänge): Schutzkontakt-...,anschlusstyp a bzw eingange schutzkontakt gumm...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1671025477624...",NaN,NaN,NaN,..

In [189]:
import numpy

df_prod_no_parsed = df_prod_vectorized.iloc[:, :7]
df_prod_no_parsed = df_prod_no_parsed.merge(df_prod_no_parsed, how="cross")
# cosine_similarity(df_prod_no_parsed["Product_title_tfidf"])
# df_prod_no_parsed["Product_title_tfidf"].to_numpy()
# numpy.array(df_prod_no_parsed["Product_title_tfidf"])

# iterate over rows, calculate cosine similarities  and add columns

C:\Users\VANCLA~1\AppData\Local\Temp/ipykernel_10976/1328682951.py:7: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



array([1, 2, (3, 4)], dtype=object)